In [1]:
import theano
import numpy as np
import sys
import pandas as pd
import scipy
from scipy.stats import spearmanr

%matplotlib inline
import matplotlib.pyplot as plt

C:\Users\kigalab\Anaconda3\envs\DeepSeq\lib\site-packages\theano\gpuarray\dnn.py:184: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to a version >= v5 and <= v7.
  warnings.warn("Your cuDNN version is more recent than "
Using cuDNN version 7300 on context None
Mapped name None to device cuda: GeForce GTX 1050 Ti (0000:01:00.0)


We will walk through the basic functions of loading up a model and predicting the effects of mutations.

# Downloading pretrained parameters

Please first download the pretrained parameters with download_pretrained.sh. 

# Loading the model

In [2]:
sys.path.insert(0, "../DeepSequence")

import model
import helper_onemutation_elbo
import train

# Mutation effect prediction

Mutation effect prediction helper functions are always with respect to the focus sequence of the alignment. We can ask for a prediction of mutation effect individually.

For reliable mutation effect prediction results, we recommend taking Monte Carlo 500-2000 samples from the model (with the N_pred_iterations parameter).

We can predict the effects of single, double, triple mutants, etc. Mutations are organized as a list of tuples, where the tuples are (uniprot position, wt amino acid, mutant amino acid).

# PABP

In [3]:
data_params = {"alignment_file":"datasets/BLAT_ECOLX_hmmerbit_plmc_n5_m30_f50_t0.2_r24-286_id100_b105.a2m"}
blat_data_helper = helper_onemutation_elbo.DataHelper(
                alignment_file=data_params["alignment_file"],
                working_dir=".",
                calc_weights=False
                )

model_params = {
        "batch_size"        :   100,
        "encode_dim_zero"   :   1500,
        "encode_dim_one"    :   1500,
        "decode_dim_zero"   :   100,
        "decode_dim_one"    :   500,
        "n_patterns"        :   4,
        "n_latent"          :   30,
        "logit_p"           :   0.001,
        "sparsity"          :   "logit",
        "encode_nonlin"     :   "relu",
        "decode_nonlin"     :   "relu",
        "final_decode_nonlin":  "sigmoid",
        "output_bias"       :   True,
        "final_pwm_scale"   :   True,
        "conv_pat"          :   True,
        "d_c_size"          :   40
        }

blat_vae_model   = model.VariationalAutoencoder(blat_data_helper,
    batch_size              =   model_params["batch_size"],
    encoder_architecture    =   [model_params["encode_dim_zero"],
                                model_params["encode_dim_one"]],
    decoder_architecture    =   [model_params["decode_dim_zero"],
                                model_params["decode_dim_one"]],
    n_latent                =   model_params["n_latent"],
    n_patterns              =   model_params["n_patterns"],
    convolve_patterns       =   model_params["conv_pat"],
    conv_decoder_size       =   model_params["d_c_size"],
    logit_p                 =   model_params["logit_p"],
    sparsity                =   model_params["sparsity"],
    encode_nonlinearity_type       =   model_params["encode_nonlin"],
    decode_nonlinearity_type       =   model_params["decode_nonlin"],
    final_decode_nonlinearity      =   model_params["final_decode_nonlin"],
    output_bias             =   model_params["output_bias"],
    final_pwm_scale         =   model_params["final_pwm_scale"],
    working_dir             =   ".")

print ("Model built")

Encoding sequences
Neff = 8355.0
Data Shape = (8355L, 253L, 20L)
self.decoder_architecture = [100, 500]
len(self.decoder_architecture) =  2
W_mu =  W_conv-mu
W_log_sigma = W_conv-log_sigma
W_conv = Elemwise{add,no_inplace}.0
self.final_output_size =  2000
self.seq_len 253
self.conv_decoder_size =  40
W_conv = Elemwise{add,no_inplace}.0
W_out = dot.0
KLD_latent =  Elemwise{mul,no_inplace}.0
all_likelihood_components =  <theano.compile.function_module.Function object at 0x000000009DFBB348>
Model built


Load up the parameters of a pretrained model in the 'params' folder.

In [4]:
file_prefix = "BLAT_ECOLX"
blat_vae_model.load_parameters(file_prefix=file_prefix)
print ("Parameters loaded")

{'W_encode_1': array([[ 0.00016344, -0.02887315,  0.0011777 , ...,  0.00500909,
         0.00388667,  0.008266  ],
       [-0.03708944,  0.01133811,  0.00081187, ...,  0.02222004,
        -0.00105152,  0.0359632 ],
       [-0.03951193, -0.01450212,  0.0258178 , ..., -0.03918116,
         0.00811642,  0.00564051],
       ...,
       [ 0.01916475, -0.00926459, -0.0239361 , ..., -0.02484108,
        -0.01363433,  0.0256789 ],
       [ 0.01440223, -0.01741994,  0.01294102, ...,  0.01860227,
         0.00792491,  0.00689663],
       [-0.00736586, -0.01432804,  0.02731984, ..., -0.02513914,
        -0.02034481,  0.01024263]], dtype=float32), 'W_encode_0': array([[-0.00949332, -0.00135953, -0.02918151, ..., -0.01331161,
         0.02111382, -0.03185128],
       [ 0.01163213, -0.01693833,  0.02271285, ...,  0.03104344,
        -0.01285565, -0.0380985 ],
       [-0.02726637,  0.00437439,  0.01698588, ...,  0.01093476,
         0.00232123,  0.00995781],
       ...,
       [-0.03015266, -0.017175

{'W_encode_1': array([[-6.e-45, -6.e-45, -6.e-45, ...,  6.e-45,  6.e-45,  6.e-45],
       [-6.e-45, -6.e-45, -6.e-45, ...,  6.e-45,  6.e-45,  6.e-45],
       [-6.e-45, -6.e-45, -6.e-45, ...,  6.e-45,  6.e-45,  6.e-45],
       ...,
       [-6.e-45, -6.e-45, -6.e-45, ...,  6.e-45,  6.e-45,  6.e-45],
       [-6.e-45, -6.e-45, -6.e-45, ...,  6.e-45,  6.e-45,  6.e-45],
       [-6.e-45, -6.e-45, -6.e-45, ...,  6.e-45,  6.e-45,  6.e-45]],
      dtype=float32), 'W_encode_0': array([[-6.e-45, -6.e-45, -6.e-45, ..., -6.e-45, -6.e-45, -6.e-45],
       [ 0.e+00,  0.e+00,  0.e+00, ...,  0.e+00,  0.e+00,  0.e+00],
       [-6.e-45, -6.e-45, -6.e-45, ..., -6.e-45, -6.e-45, -6.e-45],
       ...,
       [-6.e-45, -6.e-45, -6.e-45, ..., -6.e-45, -6.e-45, -6.e-45],
       [-6.e-45, -6.e-45,  6.e-45, ..., -6.e-45, -6.e-45, -6.e-45],
       [-6.e-45, -6.e-45, -6.e-45, ..., -6.e-45, -6.e-45, -6.e-45]],
      dtype=float32), 'final_pwm_scale-log_sigma': array([-0.0006972], dtype=float32), 'b_encode_0': array(

{'W_encode_1': array([[7.e-43, 7.e-43, 7.e-43, ..., 7.e-43, 7.e-43, 7.e-43],
       [7.e-43, 7.e-43, 7.e-43, ..., 7.e-43, 7.e-43, 7.e-43],
       [7.e-43, 7.e-43, 7.e-43, ..., 7.e-43, 7.e-43, 7.e-43],
       ...,
       [7.e-43, 7.e-43, 7.e-43, ..., 7.e-43, 7.e-43, 7.e-43],
       [7.e-43, 7.e-43, 7.e-43, ..., 7.e-43, 7.e-43, 7.e-43],
       [7.e-43, 7.e-43, 7.e-43, ..., 7.e-43, 7.e-43, 7.e-43]],
      dtype=float32), 'W_encode_0': array([[7.e-43, 7.e-43, 7.e-43, ..., 7.e-43, 7.e-43, 7.e-43],
       [0.e+00, 0.e+00, 0.e+00, ..., 0.e+00, 0.e+00, 0.e+00],
       [7.e-43, 7.e-43, 7.e-43, ..., 7.e-43, 7.e-43, 7.e-43],
       ...,
       [7.e-43, 7.e-43, 7.e-43, ..., 7.e-43, 7.e-43, 7.e-43],
       [7.e-43, 7.e-43, 7.e-43, ..., 7.e-43, 7.e-43, 7.e-43],
       [7.e-43, 7.e-43, 7.e-43, ..., 7.e-43, 7.e-43, 7.e-43]],
      dtype=float32), 'final_pwm_scale-log_sigma': array([1.0812898e-05], dtype=float32), 'b_encode_0': array([7.e-43, 7.e-43, 7.e-43, ..., 7.e-43, 7.e-43, 7.e-43], dtype=float32)

Parameters loaded


In [ ]:
BLAT_custom_matr_mutant_name_list, BLAT_custom_matr_delta_elbos \
    = blat_data_helper.custom_mutant_matrix_mutation_elbo("mutations/BLAT_ECOLX_Ranganathan2015.csv", \
                                            blat_vae_model, N_pred_iterations=500)

In [ ]:
for i in range(len(BLAT_custom_matr_mutant_name_list)):
    print (BLAT_custom_matr_mutant_name_list[i], BLAT_custom_matr_delta_elbos[i])

Let's also make a quick function to calculate the spearman rho from a mutation file.

In [11]:
file = open("BLATmutation_elbo.csv",'w')

In [12]:
for i in range(len(BLAT_custom_matr_mutant_name_list)):
    file.write(BLAT_custom_matr_mutant_name_list[i]+","+str(BLAT_custom_matr_delta_elbos[i])+"\n")

In [13]:
file.close()